LOADING

In [ ]:
# Ruta del PDF.
ruta_IVDR = r"C:/Users/Alejandro/Desktop/IVDR.pdf"

from langchain_community.document_loaders import PDFPlumberLoader
# Cargar el documento PDF y eliminar las dos últimas páginas sin contenido relevante (índice).
IVDR = PDFPlumberLoader(ruta_IVDR).load()
documento_sin_final = IVDR[:-2]

CHUNKING

In [ ]:
from langchain_core.documents import Document
# Combinar contenido (datos + metadatos) de las páginas en un solo documento. Sino, al dividir por capítulos y artículos, se cortaría en cada final de página.
contenido_combinado = "\n\n".join(doc.page_content for doc in documento_sin_final)
documento_combinado = Document(page_content=contenido_combinado, metadata=IVDR[0].metadata)

import re
# Expresión regular para detectar capítulos del estilo "CAPÍTULO XX".
patron_capitulos = r"(CAPÍTULO\s+[IVXLCDM\d]+(?:\n[^\n]+)?)"
division_capitulos = re.split(patron_capitulos, documento_combinado.page_content)

# Construcción de capítulos como documentos. Al principio se añade el capítulo 0 (Introducción). Se coge el titulo del capítulo y su contenido.
capitulos = [("CAPÍTULO 0 - Introducción", division_capitulos[0].strip())] if division_capitulos[0].strip() else []

for i, numero_capitulo in enumerate(range(1, len(division_capitulos) // 2 + 1), start=1):
    titulo_capitulo = division_capitulos[i * 2 - 1].strip().replace('\n', ' ')
    titulo = f"CAPÍTULO {numero_capitulo} - {titulo_capitulo}"
    contenido = f"{titulo_capitulo}\n\n{division_capitulos[i * 2].strip()}"
    capitulos.append((titulo, contenido))

# Crear documentos por capítulo.
documento_por_capitulos = [Document(page_content=contenido, metadata={"titulo": titulo}) for titulo, contenido in capitulos]

from langchain_text_splitters import TextSplitter
# Clase personalizada para dividir en artículos. Divide el texto en artículos, incluyendo la introducción si existe.
class ArticleTextSplitter(TextSplitter):
    
    def split_text(self, text: str):
        pattern = r"(Artículo\s+\d+\n\b[^\n]*)"
        split_text = re.split(pattern, text)

        articulos = []

        # Si hay texto antes del primer 'Artículo', se considera introducción.
        if split_text[0].strip():
            articulos.append({
                "titulo": "INTRODUCCIÓN",
                "contenido": split_text[0].strip()
            })

        for i in range(1, len(split_text), 2):
            titulo_articulo = split_text[i].strip().replace('\n', ' ')
            contenido = f"{titulo_articulo}\n\n{split_text[i + 1].strip()}" if i + 1 < len(split_text) else titulo_articulo

            articulos.append({
                "titulo": titulo_articulo,
                "contenido": contenido
            })

        return articulos

# División por artículos.
documento_por_articulos = ArticleTextSplitter()

from langchain_text_splitters import RecursiveCharacterTextSplitter
# Fragmentación en chunks para dividir artículos largos.
chunk_splitter = RecursiveCharacterTextSplitter(
    chunk_size=10000,
    chunk_overlap=500,
    separators=["\n\n", "\n", ".", " "]
)

# Lista final de chunks: Se divide por capítulos, luego por artículos y finalmente en fragmentos más pequeños.
chunks = []

for capitulo in documento_por_capitulos:
    articulos = documento_por_articulos.split_text(capitulo.page_content)

    for articulo in articulos:
        sub_chunks = chunk_splitter.split_text(articulo["contenido"])

        for i, sub_chunk in enumerate(sub_chunks):
            chunks.append(
                Document(
                    page_content=sub_chunk,
                    metadata={
                        "capitulo": capitulo.metadata["titulo"],
                        "artículo": articulo["titulo"],
                        "fragmento": i  #Índice del fragmento dentro del artículo.
                    }
                )
            )

---LIMPIEZA (OPCIONAL)

In [4]:
import shutil

# 📌 1. Eliminar por completo la base de datos anterior
persist_directory = "vectorstore"
shutil.rmtree(persist_directory, ignore_errors=True)  # Borra la carpeta

EMBEDDING

In [5]:
# Crear embeddings "sentence-transformers/all-MiniLM-L6-v2" vs "BAAI/llm-embedder".
from langchain_huggingface import HuggingFaceEmbeddings

modelo_embedding = HuggingFaceEmbeddings(model_name="BAAI/llm-embedder")

VECTORSTORE

In [ ]:
# Crear la base de datos vectorial (vectorstore).
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(
    documents=chunks,
    collection_name="BAAI_RAG",
    embedding=modelo_embedding)  #Si da algun error, borrarla con vectorstore.delete_collection()

---PRUEBAS RETRIEVE (OPCIONAL)

In [ ]:
pregunta = "Clasificación de los productos sanitarios"
#-------PRUEBA DE COMO NOS SALDRIAN LAS RESPUESTAS-----------
# Recuperar los documentos de la vectorstore más relevantes por similaridad
test_docs = vectorstore.similarity_search_with_score(pregunta, k=10)

# Ordenar manualmente por score de menor a mayor (los más relevantes primero)
test_docs.sort(key=lambda x: x[1])

# Mostrar los documentos ordenados con sus metadatos y scores
for i, (doc, score) in enumerate(test_docs):
    articulo = doc.metadata.get("artículo", "Desconocida")  
    title = doc.metadata.get("capitulo", "Sin título") 
    print(f"Chunk {i+1} (Score: {score}, Artículo: {articulo}, Título: {title}):\n{doc.page_content[:1300]}\n")
#--------FIN DE LA PRUEBA, ELEGIR EL NUMERO DE K QUE MAS NOS INTERESE-----------

RETRIEVING

In [ ]:
# La base de datos vectorial se utilizara como recuperador (retriever) de documentos.
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

HyDE

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

# Se define un prompt para HyDE (Hyphotetical Document Embedding).
HyDE_plantilla = """Genera un texto regulatorio breve que podría responder a la siguiente consulta. 
La respuesta debe ser concisa y estructurada en un máximo de 6 frases.
Pregunta: {pregunta}  
Texto:"""
prompt_HyDE = ChatPromptTemplate.from_template(HyDE_plantilla)

# Se define el LLM local a traves de Ollama que se utilizará para generar el documento hipotético y la respuesta a la pregunta.
llm = ChatOllama(model="llama3.2:3b", temperature=0) # t = 0 para respuestas más precisas y menos creativas.

# Cadena que une el prompt, el LLM y el parser de salida del documento hipotético.
cadena_generacion_HyDE = ( prompt_HyDE | llm | StrOutputParser())

# Invocar la cadena de generación de documento HyDE solo con la pregunta.
cadena_generacion_HyDE.invoke({"pregunta":pregunta})

# Se define una cadena de recuperacion de documentos. 
cadena_recuperacion = cadena_generacion_HyDE | retriever 

# Invocar la cadena final solo con la pregunta.
documentos_recuperados = cadena_recuperacion.invoke(pregunta)

# Recuperar documentos relevantes
documentos_recuperados = retriever.invoke(pregunta)

# Función para concatenar documentos separándolos por dos saltos de línea.
def concatenar_docs(documents):
    return "\n\n".join([doc.page_content for doc in documents])

# Se concatenan los documentos antes de pasarlos al LLM.
docs_concatenados = concatenar_docs(documentos_recuperados)

GENERACIÓN

In [ ]:
# ETAPA DE GENERACIÓN. Cadena de prompt(docs + pregunta) -> LLM -> Respuesta en string
# Se define el prompt 
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    template="""Eres un asistente experto en regulación sanitaria que responde preguntas únicamente utilizando el contenido proporcionado en los documentos normativos.

Instrucciones importantes:
- Solo puedes usar la información incluida en los documentos para responder. No inventes ni asumas información externa.
- Si algún fragmento menciona un artículo o capítulo, debe citarse también en la respuesta.
- Si los documentos no contienen la información suficiente para responder, simplemente indica que no dispones de esa información.
- La respuesta debe ser clara, directa y tener como máximo cuatro líneas.

Pregunta: {pregunta}

Documentos recuperados:
{docs_concatenados}

Respuesta:""",
    input_variables=["pregunta", "docs_concatenados"],
)

# Cadena final que une el prompt, el LLM y el parser de salida.
cadena_generacion_rag = (prompt | llm | StrOutputParser())

# Invocar la cadena final con los documentos formateados y la pregunta.
cadena_generacion_rag.invoke({"docs_concatenados":docs_concatenados,"pregunta":pregunta})